In [1]:
import pandas as pd
import numpy as np

evidence = pd.read_json("/content/drive/MyDrive/Colab Notebooks/NLP project/FactChecker_NLP/data/evidence.json",orient='index')
train = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP project/FactChecker_NLP/data/train-claims.json')



In [2]:
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.1 MB/s eta 0:00:00



Using spacy to extract nouns chunks


In [6]:
import spacy

# Load the spacy model
nlp = spacy.load('en_core_web_sm')

# Get the backend you are using (cupy if you have a CUDA enabled GPU and thinc is installed, otherwise numpy)
backend = spacy.prefer_gpu()
print('Using GPU:', backend)

# Process whole documents
evidences = evidence.iloc[:, 0].tolist()
test = evidences[0:100]
claims = train.iloc[0].tolist()

# Process the text
for sentence in claims[0:20]:
  print("original: "+sentence)
  doc = nlp(sentence)

  # Analyze syntax
  nouns = [chunk.text for chunk in doc.noun_chunks]
  print('Nouns:', nouns)

  strong_words = [token.text for token in doc if token.pos_ in ['PROPN','NOUN']]
  print(strong_words)


  # Named entities, phrases and concepts
  for entity in doc.ents:
      print(entity.text, entity.label_)
  
  print("=====================")


Using GPU: False
original: Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.
Nouns: ['no scientific evidence', 'CO2', 'a pollutant, higher CO2 concentrations', 'ecosystems', 'more plant and animal life']
['evidence', 'CO2', 'CO2', 'concentrations', 'ecosystems', 'plant', 'animal', 'life']
original: El Niño drove record highs in global temperatures suggesting rise may not be down to man-made emissions.
Nouns: ['El Niño', 'record highs', 'global temperatures', 'rise', 'man-made emissions']
['El', 'Niño', 'highs', 'temperatures', 'rise', 'man', 'emissions']
El Niño ORG
original: In 1946, PDO switched to a cool phase.
Nouns: ['PDO', 'a cool phase']
['PDO', 'phase']
1946 DATE
PDO ORG
original: Weather Channel co-founder John Coleman provided evidence that convincingly refutes the concept of anthropogenic global warming.
Nouns: ['co', '-', 'evidence', 'that', 'the concept', 'anthropogenic g

In [4]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
import random

# 对你的evidence数据进行编码
ran_evidences = random.sample(evidences, 10000)
  # 用你的evidence数据替换这里
evidence_embeddings = []

# 初始化模型和tokenizer
tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
model = model.to(device)

# 对你的evidence数据进行编码
ran_evidences = random.sample(sentences, 10000)
evidence_embeddings = []

batch_size = 32  # 设置你的批量大小

for i in range(0, len(ran_evidences), batch_size):
    print("Encoding batch {} of {}".format(i // batch_size, len(ran_evidences) // batch_size))
    batch = ran_evidences[i:i+batch_size]
    inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        output = model(**inputs)
    evidence_embeddings.extend(output.pooler_output.detach().cpu().numpy())

# 将列表转化为numpy数组
evidence_embeddings = np.vstack(evidence_embeddings)

# 使用numpy的argsort函数来获取排序后的索引
def retrieve_top_k(query_embedding, evidence_embeddings, k=4):
    distances = np.linalg.norm(evidence_embeddings - query_embedding, axis=1)
    indices = np.argsort(distances)
    return indices[:k]

# 对一个claim进行编码并检索最相关的evidence
claim = "hello world"  # 用你的claim替换这里
inputs = tokenizer(claim, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
with torch.no_grad():
    output = model(**inputs)

claim_embedding = output.pooler_output.detach().cpu().numpy()

# 进行检索，返回最相关的4个evidence
indices = retrieve_top_k(claim_embedding, evidence_embeddings, k=4)

# 输出检索到的evidence
for i in indices:
    print(ran_evidences[i])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


NameError: ignored